In [1]:
import numpy as np
import pandas as pd


In [43]:
scores_oct = pd.read_csv('nbaoct.csv')
scores_nov = pd.read_csv('nbanov.csv')
scores_dec = pd.read_csv('nbadec.csv')

In [60]:
scores = pd.concat([scores_oct,scores_nov,scores_dec])
scores = scores[~scores.Date.isnull()]
games = scores.shape[0]
games

378

In [91]:
scores['winner'] = np.where(scores['PTS']>scores['PTS.1'], scores['Visitor/Neutral'], scores['Home/Neutral'])
scores['loser'] = np.where(scores['PTS']>scores['PTS.1'], scores['Home/Neutral'], scores['Visitor/Neutral'])
scores['delta'] = abs(scores['PTS'] - scores['PTS.1'])
scores2 = scores[['winner','loser','delta']]
scores2.reset_index(drop=True,inplace=True)

In [92]:
teams = scores['winner'].unique()
results = scores2['delta'].values
teams

array(['Milwaukee Bucks', 'Golden State Warriors', 'Charlotte Hornets',
       'Chicago Bulls', 'New York Knicks', 'Washington Wizards',
       'Memphis Grizzlies', 'Minnesota Timberwolves',
       'Philadelphia 76ers', 'San Antonio Spurs', 'Utah Jazz',
       'Sacramento Kings', 'Denver Nuggets', 'Atlanta Hawks',
       'Miami Heat', 'Toronto Raptors', 'Brooklyn Nets',
       'Houston Rockets', 'Phoenix Suns', 'Cleveland Cavaliers',
       'Indiana Pacers', 'Dallas Mavericks', 'Portland Trail Blazers',
       'Boston Celtics', 'Orlando Magic', 'Los Angeles Lakers',
       'New Orleans Pelicans', 'Los Angeles Clippers',
       'Oklahoma City Thunder', 'Detroit Pistons'], dtype=object)

In [93]:
M = np.zeros((games,30))

for i,m in enumerate(M):
    
    w = scores2.loc[i,'winner']
    l = scores2.loc[i,'loser']
    locw = np.where(teams==w)
    locl = np.where(teams==l)
    M[i,locw] = 1
    M[i,locl] = -1

In [95]:
LHS = M.T@M
RHS = M.T@results
RHS[-1] = 0
LHS[-1] = 1
ranks = np.linalg.solve(LHS,RHS)



In [96]:
rankings = pd.DataFrame(teams,columns=['Team']).reset_index()
rankings['rank'] = ranks

ranked = rankings.sort_values(by='rank',ascending=False).reset_index(drop=True)
ranked = ranked[['Team','rank']]
ranked = ranked.reset_index()
ranked.columns = ['Ranking','Team','Rank_score']
ranked['Ranking'] = ranked['Ranking']+1
ranked.set_index('Ranking',inplace=True)

In [97]:
ranked

,Team,Rank_score
Ranking,,
1,Golden State Warriors,11.770453
2,Utah Jazz,8.964561
3,Phoenix Suns,6.144027
4,Cleveland Cavaliers,3.970268
5,Chicago Bulls,3.289821
6,Atlanta Hawks,3.005202
7,Miami Heat,2.885195
8,Milwaukee Bucks,2.881005
9,Brooklyn Nets,2.386823
